# WEBSITE LINK : https://jeansfirstapp0806-c00b1f941514.herokuapp.com/

# Importing libraries

In [ ]:
import pathlib
import pandas as pd
import dash
import calendar
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
from dash import Dash, html, dcc
import requests
from io import StringIO


# Creating file paths; reading data with pandas

In [ ]:

PATH = pathlib.Path(__file__).parent
DATA_PATH = PATH.joinpath("data").resolve()

# Pizza tables from SQL, Rain from Excel
df = pd.read_csv(DATA_PATH.joinpath('pizzatables1.csv'))
rain = pd.read_csv(DATA_PATH.joinpath('RAIN_FALL_2015_-_Sheet1.csv'))

# Using Dash to create web layout

In [ ]:
# Define the desired order of the days
day_order = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

In [ ]:
# Define the app layout
app = Dash(__name__)
server= app.server
app.layout = html.Div([
    html.H1("Mamma Mia Pizza Data"),
    dcc.Tabs(id="tabs", value='tab-1', children=[
        dcc.Tab(label='Peak hours.', value='tab-1'),
        dcc.Tab(label='How rain affects orders.', value='tab-2'),
        dcc.Tab(label='What pizza types sell most.', value='tab-3'),
        dcc.Tab(label='How much ingredients are being used.', value='tab-4'),
        dcc.Tab(label='Analyzing size issues.', value='tab-5')
    ]),
    html.Div(id='content')
])


# Define the callback to update the content based on the selected tab
@app.callback(
    Output('content', 'children'),
    Input('tabs', 'value')
)

# Each tab displays correct graph on web

In [ ]:
def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            html.H2('Order amounts by hour'),
            dcc.Dropdown(
                id='day-dropdown',
                options=[{'label': day, 'value': day} for day in day_order],
                value='Sunday',
                style={'margin-bottom': '10px'}  # Add margin at the bottom of the dropdown
            ),
            html.Div([
                dcc.Graph(
                    id='heatmap-graph',
                    figure=update_heatmap('Sunday'),  # Default day is Sunday
                    style={'margin-bottom': '120px'}  # Add margin at the bottom of the heatmap
                ),
                html.Hr(style={'border-top': '6px solid rgba(0, 0, 0, 0.5)', 'margin': '10px 0'})  # Add black line with margin
            ], style={'margin-bottom': '120px'}),  # Add margin between line and treemap
            html.H2('Best days and hours'),
            dcc.Graph(
                id='treemap-graph',
                figure=update_treemap(),
                style={'margin-bottom': '250px'}  # Add margin at the bottom of the treemap
            )
        ])
    elif tab == 'tab-2':
        return html.Div([
            html.H2('Rainfall and Orders'),
            dcc.Graph(
                id='subplot-graph',
                figure=update_subplot_graph()
            )
        ])
    elif tab == 'tab-3':
        return html.Div([
            html.H2('Pizza Types'),
            dcc.Graph(
                id='histogram',
                figure=update_histogram()
            )
        ])
    elif tab == 'tab-4':
        return html.Div([
            html.H2('Ingredients'),
            dcc.Graph(
                id='ingredient-histogram',
                figure=update_ingredient_histogram()
            )
        ])
    elif tab == 'tab-5':
        return html.Div([
            html.H2('Pizza Sizes'),
            dcc.Graph(
                id='pie-chart',
                figure=update_pie_chart()
            ),
            dcc.Graph(
                id='bar-chart',
                figure=update_bar_chart()
            )
        ])

# Connect dash components with figures and return visual

# Heatmap

In [ ]:

@app.callback(
    Output('heatmap-graph', 'figure'),
    Input('day-dropdown', 'value')
)
def update_heatmap(day):
    filtered_df = df[df['day'] == day]
    pizza_type_counts = filtered_df['pizza_type_id'].value_counts()
    popular_pizza_types = pizza_type_counts.nlargest(15).index.tolist()
    filtered_df = filtered_df[filtered_df['pizza_type_id'].isin(popular_pizza_types)]
    filtered_df['hour'] = pd.to_numeric(filtered_df['hour'])  # Convert hour column to numeric
    filtered_df = filtered_df.sort_values('hour')  # Sort DataFrame by hour column
    heatmap_fig = px.density_heatmap(filtered_df, x='hour', y='pizza_type_id', z='quantity', color_continuous_scale='RdYlGn')
    heatmap_fig.update_layout(xaxis={'type': 'category'})  

    return heatmap_fig

# Treemap

In [ ]:
def update_treemap():
    grouped_df = df.groupby(['day', 'hour']).sum(numeric_only=True).reset_index()
    grouped_df['day'] = pd.Categorical(grouped_df['day'])
    grouped_df = grouped_df.sort_values(['day', 'hour'])
    
    treemap_fig = px.treemap(grouped_df, path=['day', 'hour'], values='quantity',color= 'quantity', color_continuous_scale='RdYlGn')
    treemap_fig.update_traces(
        selector=dict(type='treemap')
    )
    treemap_fig.update_layout(
        margin=dict(l=10, r=10, t=40, b=10),
        coloraxis_showscale=False
    )
    
    return treemap_fig

# Rainfall

In [ ]:
def update_subplot_graph():
    results = df.groupby('month').sum()
    months = list(range(1, 13))
    days_in_month = [calendar.monthrange(2023, month)[1] for month in months]
    adjusted_quantity = results['quantity'] / days_in_month * 30
    rainfall = rain.groupby('month').sum()

    # Create the subplot fig
    fig = sp.make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=months, y=adjusted_quantity, name='Orders', line=dict(color='green')), secondary_y=False)
    fig.add_trace(go.Scatter(x=months, y=rainfall['rainfall'], name='Rainfall', line=dict(color='blue')), secondary_y=True)
    fig.update_layout(
        xaxis=dict(title='Month', dtick=1),
        yaxis=dict(
            title='Orders',
            titlefont=dict(color='black'),
            tickfont=dict(color='green')
        ),
        yaxis2=dict(
            title='Rainfall in inches',
            titlefont=dict(color='black'),
            tickfont=dict(color='blue'),
            anchor='x',
            overlaying='y',
            side='right'
        ),
        legend=dict(x=0, y=1.2, orientation='h')
    )

    return fig

# Pizza Types Histogram

In [ ]:
def update_histogram():
    sorted_df = df.sort_values('quantity', ascending=False)
    histogram_fig = px.histogram(sorted_df, x='pizza_type_id', y='quantity', color='category', barmode='group')
    
    return histogram_fig

# Ingredients

In [ ]:
# Needed to split ingredients by , and remove white space to make each ingredient be seperate.
# Use to evaluate inventory management
def update_ingredient_histogram():
    ingredient_df = df.copy()
    ingredient_df['individual_ingredients'] = ingredient_df['ingredients'].str.split(',')
    ingredient_df['individual_ingredients'] = ingredient_df['individual_ingredients'].apply(lambda x: [ingredient.strip() for ingredient in x])
    ingredient_df = ingredient_df.explode('individual_ingredients')
    ingredient_sums = ingredient_df.groupby(['pizza_type_id', 'individual_ingredients'])['quantity'].sum().reset_index()
    ingredient_sums = ingredient_sums.sort_values(by='quantity', ascending=False)

    unique_ingredients = ingredient_sums.groupby('individual_ingredients')['pizza_type_id'].nunique().reset_index()
    unique_ingredients = unique_ingredients[unique_ingredients['pizza_type_id'] == 1]

    ingredient_sums['unique_ingredient'] = ingredient_sums['individual_ingredients'].isin(unique_ingredients['individual_ingredients'])

    ingredient_sums_unique = ingredient_sums[~ingredient_sums['unique_ingredient']]
    ingredient_sums_unique = ingredient_sums_unique.sort_values(by='quantity', ascending=False)

    histogram_ingredient_fig = px.bar(
        ingredient_sums,
        x='individual_ingredients',
        y='quantity',
        color='unique_ingredient',
        labels={'individual_ingredients': 'Ingredient', 'quantity': 'Summed Quantity'},
    )

    histogram_ingredient_fig.update_traces(marker=dict(line=dict(color='white', width=0.5)))

    histogram_ingredient_fig.update_layout(coloraxis=dict(colorscale=[[0, '#999999'], [1, '#FF0000']]))

    histogram_ingredient_fig.update_xaxes(categoryorder='array', categoryarray=ingredient_sums_unique['individual_ingredients'])

    histogram_ingredient_fig.update_layout(height=500)

    return histogram_ingredient_fig

# The Greek pie chart

In [ ]:
# I have the pie chart to evaluate if XL and XXL are viable options
def update_pie_chart():
    filtered_df = df[df['pizza_type_id'] == 'the_greek']
    size_quantity = filtered_df.groupby('size')['quantity'].sum().reset_index()

    pie_chart_fig = go.Figure(data=go.Pie(labels=size_quantity['size'], values=size_quantity['quantity']))

    pie_chart_fig.update_layout(title='Pizza Sizes Distribution for "The Greek"',
                                showlegend=True)

    return pie_chart_fig

# Sizes Bar chart

In [ ]:
# I've removed the greek, brie carre, big meats, and five cheese because they would represent outlier
# Because of their size options
def update_bar_chart():
    filtered_df_pie = df[df['pizza_type_id'] == 'the_greek']
    filtered_df_bar = df.copy()

    excluded_sizes = ['XL', 'XXL']
    filtered_df_bar = filtered_df_bar[~filtered_df_bar['size'].isin(excluded_sizes)]

    excluded_pizza_types = ['big_meat', 'brie_carre', 'five_cheese']
    filtered_df_bar = filtered_df_bar[~filtered_df_bar['pizza_type_id'].isin(excluded_pizza_types)]

    size_quantity_bar = filtered_df_bar.groupby('size')['quantity'].sum().reset_index().sort_values('quantity')

    bar_chart_fig = go.Figure(data=go.Bar(x=size_quantity_bar['size'], y=size_quantity_bar['quantity']))

    bar_chart_fig.update_layout(title='Summed Quantity for Each Pizza Size (Excluding XL, XXL, and Specific Types)',
                                xaxis_title='Size',
                                yaxis_title='Summed Quantity')

    return bar_chart_fig




# Run

In [ ]:

if __name__ == '__main__':
    app.run_server()